<a href="https://colab.research.google.com/github/tanatet8/Colab_Script/blob/main/facebook_caption_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ติดตั้ง selenium และ chromium สำหรับใช้งาน browser แบบอัตโนมัติ
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# เพิ่ม path ให้ Python หา chrome driver เจอ
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

# สร้าง options สำหรับ Chrome browser
chrome_options = Options()
chrome_options.add_argument("--headless")  # ไม่แสดงหน้าต่าง browser
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--no-sandbox")

# เปิด browser โดยใช้ options ที่ตั้งไว้
driver = webdriver.Chrome(options=chrome_options)

In [ ]:
# เปิดหน้าเพจ Facebook ที่เราจะดึงข้อมูลโพสต์ (Public เท่านั้น)
driver.get("https://www.facebook.com/people/เขียนไว้ให้เธอ/100044623680446/?sk=photos")

# รอโหลดข้อมูลประมาณ 5 วินาที
import time
time.sleep(5)

In [ ]:
# เลื่อนหน้าจอลงทีละช่วง เพื่อให้ Facebook โหลดโพสต์เพิ่ม
scroll_pause_time = 2
last_height = driver.execute_script("return document.body.scrollHeight")

for i in range(10):  # สามารถเพิ่มรอบได้หากต้องการดึงโพสต์ลึกกว่านี้
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [ ]:
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

captions = []

# หาโพสต์ทั้งหมดจากลิงก์ที่เป็น photo
posts = driver.find_elements(By.XPATH, "//a[contains(@href, '/photo.php')]")

print(f"📸 พบโพสต์ทั้งหมด: {len(posts)}")

# เปิดแต่ละโพสต์เพื่อดึงข้อความ
for i, post in enumerate(posts):
    try:
        post_url = post.get_attribute('href')
        if post_url:
            # เปิดแท็บใหม่ไปยังโพสต์
            driver.execute_script("window.open(arguments[0]);", post_url)
            driver.switch_to.window(driver.window_handles[1])
            time.sleep(3)

            try:
                # ดึงข้อความแคปชันจากโพสต์
                caption = driver.find_element(By.XPATH, "//div[@data-ad-comet-preview='message']").text
                captions.append(caption)
                print(f"✅ โพสต์ที่ {i+1}: {caption[:60]}...")
            except NoSuchElementException:
                print(f"⛔️ ไม่มีข้อความในโพสต์ที่ {i+1}")

            # ปิดแท็บ แล้วกลับมาที่แท็บหลัก
            driver.close()
            driver.switch_to.window(driver.window_handles[0])

    except Exception as e:
        print(f"❌ Error ที่โพสต์ {i+1}: {e}")
        continue

# ปิด browser หลังเสร็จ
driver.quit()

In [ ]:
# แสดงแคปชันทั้งหมดที่เก็บมาได้
for i, text in enumerate(captions, 1):
    print(f"\n📌 โพสต์ที่ {i} \n{text}\n{'-'*50}")